In [ ]:
import identification_py2 as ob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import control as con
import glob #for returning files having the specified path extension
import statistics as stats
import os #checking for empty file
%pylab inline

###### Passing all the data into arrays 

In [ ]:
task_first        = sorted(glob.glob('step_log_new/*/*task*.log')) #corresponds to .log files that has data related to the first position
control_first     = sorted(glob.glob('step_log_new/*/*control*.log'))
task_remaining    = sorted(glob.glob('step_log_new/*/*task*.log.*')) #corresponds to remaining log.'n' files
control_remaining = sorted(glob.glob('step_log_new/*/*control*.log.*'))
task              = sorted(task_first + task_remaining) #set of all task_velocity logs
control           = sorted(control_first + control_remaining) #set of all control logs
observations      = len(task_first) #total number of experiments conducted/observations taken
positions         = int(len(task) / observations) #number of points in the given task space
task_full         = [] #A task_velocity list whose each element is a list of similar log files i.e from the same position
control_full      = [] #A control_output list whose each element is a list of similar log files i.e from the same position

for i in range(0, positions):
    task_full.append([])
    control_full.append([])
    for j in range(0, observations):
        task_full[i].append(task[i + (j * positions)])
        control_full[i].append(control[i + (j * positions)])

count = 0 #counter that returns the number of empty files
for i in range(0, positions):
    for j in range(0, observations):
        if os.stat(task_full[i][j]).st_size == 0:
            count = count + 1
            
for i in range(0, positions):
    for j in range(0, observations-count):
        if os.stat(task_full[i][j]).st_size == 0:
            del(task_full[i][j])
            del(control_full[i][j])
            
# Reading all the data into a dataframe array
df_ist_soll = []
for i in range(0, positions):
    df_ist_soll.append([])
    for j in range(0, observations):
        try:
            df_ist_soll[i].append(ob.batch_read_data(control_full[i][j], task_full[i][j]))
        except:
            continue

###### Manually changing the setpoint 

In [ ]:
#for i in range(0, observations):
#    df_ist_soll[0][i].x_soll[df_ist_soll[0][i].x_soll > 0] = 0.15
#    df_ist_soll[3][i].x_soll[df_ist_soll[3][i].x_soll > 0] = 0.17

###### Displaying all the observations

In [ ]:
# The first try except code avoids the errors arising due to the already existing Overview directory. 
# The second try except code avoids the errors resulting from the plotting of the empty data file 
try:
    os.makedirs('View_Data/')
except OSError, e:
    if e.errno != os.errno.EEXIST:
        raise   
    pass

for i in range(0, positions):
    fig = plt.figure(figsize = (10,30))
    fig.suptitle('Position %s'%(i + 1), fontsize = 20, fontweight = 'bold')
    for j in range(0, observations): 
        try:
            ax = fig.add_subplot(observations, 1, j + 1)
            ax.set_title('Observation %s'%(j + 1))
            plt.tight_layout(rect=[0, 0.03, 1, 0.95])
            plt.plot(df_ist_soll[i][j]) 
        except:
            pass
    plt.savefig('View_Data/Position %s.png'%(i + 1))

###### Smoothing using Savgol filter

In [ ]:
def smoothing_sg():
    smooth_1 = [] #array having values, whoose smoothing is done according to first order 
    smooth_2 = [] #array having values, whoose smoothing is done according to second order
    
    for i in range(0, len(yout_array)):
        smooth_1.append(ob.smooth(yout_array[i], 1))
        smooth_2.append(ob.smooth(yout_array[i], 2))
    return smooth_1, smooth_2

###### PT1 Estimation 

In [ ]:
# The steady state value is calculated based on the final values of the step response. 
#In case of a faulty step response, the overall model also gets affected. 

#youto,to are the yout and t outputs from the pt1 and pt2 system
#tf and delay are the transfer functions of the output and its delay
#tdytdts is an array that contains all the above values in a sequential order
def pt1():
    to_1           = []
    tf_1           = []
    youto_1        = []
    delay_1        = []
    tdytdts_1      = []
    delay_tf_1     = []
    steady_state_1 = []
    time_constant_1 = []
    smooth_1 = smoothing_sg()[0]
    
    for i in range(0,len(smooth_1)):
        tdytdts_1.append(ob.pt1(smooth_1[i], t_array[i]))
        tf_1.append(tdytdts_1[i][0])
        youto_1.append(tdytdts_1[i][1])
        to_1.append(tdytdts_1[i][2])
        delay_1.append(tdytdts_1[i][3])
        time_constant_1.append(tdytdts_1[i][4])
        steady_state_1.append(tdytdts_1[i][5])
    return steady_state_1, time_constant_1, delay_1

###### PT2 Estimation

In [ ]:
'''PT2 modeling'''
def pt2():
    to_2            = []
    tf_2            = []
    zeta            = []
    youto_2         = []
    delay_2         = []
    tdytdts_2       = []
    delay_tf_2      = []
    steady_state_2  = []
    time_constant_2 = []
    smooth_2 = smoothing_sg()[1]
    
    try:
        for i in range(0,len(smooth_2)):
            tdytdts_2.append(ob.pt2(smooth_2[i], t_array[i]))
            tf_2.append(tdytdts_2[i][0])
            youto_2.append(tdytdts_2[i][1])
            to_2.append(tdytdts_2[i][2])
            delay_2.append(tdytdts_2[i][3])
            time_constant_2.append(tdytdts_2[i][4])
            steady_state_2.append(tdytdts_2[i][5])
            zeta.append(tdytdts_2[i][6])
    except:
        pass
    return steady_state_2, time_constant_2, delay_2, zeta

###### Plotting of ideal pt1 model from each point in the task space

In [ ]:
# Each of the timeseries in a position is modeled according to the pt1 modeling and the ideal model 
# in a position is calculated by taking the average of these individual models.


system_matrix_pt1  = [] #contains all the state space parameters of all ideal models
mean_matrix_pt1    = []
median_matrix_pt1  = []
std_matrix_pt1     = [] # std = standard deviation
var_matrix_pt1     = [] # var = variance
model_pos_pt1      = [] # model as time series for each positions
yout_full_pt1      = []
model_time_pt1     = []
model_output_pt1   = []

for i in range(0, positions):
    try:
#        xin_array, yout_array, t_array = ob.strip_multiply(df_ist_soll[i])
        xin_array, yout_array, t_array, m_factor = ob.unit_response(df_ist_soll[i])
        steady_state_1, time_constant_1, delay_1 = pt1()
        ideal_tf_pt1, ideal_model_output_pt1, ideal_model_time_pt1 = ob.ideal_pt1(steady_state_1, time_constant_1, delay_1)
    except:
        continue
    yout_full_pt1.append(yout_array)
    model_pos_pt1.append(ideal_model_output_pt1)
    mean_matrix_pt1.append(stats.mean(ideal_model_output_pt1))
    median_matrix_pt1.append(stats.median(ideal_model_output_pt1))
    std_matrix_pt1.append(stats.pstdev(ideal_model_output_pt1))
    var_matrix_pt1.append(stats.variance(ideal_model_output_pt1))
    plt.plot(ideal_model_time_pt1, ideal_model_output_pt1, label = 'position %s ideal model'%(i+1))
    plt.legend()
    plt.savefig('model_pt1.png')
    model_time_pt1.append(ideal_model_time_pt1)
    model_output_pt1.append(ideal_model_output_pt1)    
    system_matrix_pt1.append(ob.ss(ideal_tf_pt1))

###### Plotting of ideal pt2 model from each point in the task space


In [ ]:
system_matrix_pt2    = []
mean_matrix_pt2      = []
median_matrix_pt2    = []
std_matrix_pt2       = [] # std = standard deviation
var_matrix_pt2       = [] # var = variance
model_pos_pt2        = [] # model as time series for each positions
yout_full_pt2        = []
model_time_pt2   = []
model_output_pt2 = []
m_factor_array   = [] # used in model validation while plotting
for i in range(0, positions):
    try:
        #xin_array, yout_array, t_array = ob.strip_multiply(df_ist_soll[i])
        xin_array, yout_array, t_array, m_factor = ob.unit_response(df_ist_soll[i])
        steady_state_2, time_constant_2, delay_2, zeta = pt2()
        ideal_tf_pt2, ideal_model_output_pt2, ideal_model_time_pt2 = ob.ideal_pt2(steady_state_2, time_constant_2, delay_2, zeta)
    except:
        continue
    yout_full_pt2.append(yout_array)
    model_pos_pt2.append(ideal_model_output_pt2)
    mean_matrix_pt2.append(stats.mean(ideal_model_output_pt2))
    median_matrix_pt2.append(stats.median(ideal_model_output_pt2))
    std_matrix_pt2.append(stats.pstdev(ideal_model_output_pt2))
    var_matrix_pt2.append(stats.variance(ideal_model_output_pt2))
    plt.plot(ideal_model_time_pt2, ideal_model_output_pt2, label = 'position %s ideal model'%(i+1))
    plt.legend()
    plt.savefig('model_pt2.png')
    model_time_pt2.append(ideal_model_time_pt2)
    model_output_pt2.append(ideal_model_output_pt2)
    m_factor_array.append(mean(m_factor))
    system_matrix_pt2.append(ob.ss(ideal_tf_pt2))

###### Displaying statistical output of each positions in a text file

In [ ]:
quant_matrix_pt1 = []
with open("Statistical_Output_pt1.txt", "w") as text_file:
    text_file.write('###########################\n')
    text_file.write('  STATISTICAL INFORMATION  \n')
    text_file.write('###########################\n')
    for i in range(0, len(model_pos_pt1)):
        text_file.write('Position %s\n'%(i+1))
        text_file.write('Mean:%s\n' %mean_matrix_pt1[i])
        text_file.write('Median:%s\n' %median_matrix_pt1[i])
        text_file.write('Standard Deviation:{0}\n' .format(std_matrix_pt1[i]))
        text_file.write('Variance:%s\n' %var_matrix_pt1[i])
        text_file.write('Quantiles[0.25, 0.50, 0.75]:%s\n' \
              %pd.Series(model_pos_pt1[i]).quantile\
              ([.25, .5, .75]).values)
        quant_matrix_pt1.append(pd.Series(model_pos_pt1[i]).quantile([.25, .5, .75]).values)
        text_file.write('Min:%s\n' %min(model_pos_pt1[i]))
        text_file.write('Max:%s\n\n' %max(model_pos_pt1[i]))

quant_matrix_pt2 = []
with open("Statistical_Output_pt2.txt", "w") as text_file:
    text_file.write('###########################\n')
    text_file.write('  STATISTICAL INFORMATION  \n')
    text_file.write('###########################\n')
    for i in range(0, len(model_pos_pt2)):
        text_file.write('Position %s\n'%(i+1))
        text_file.write('Mean:%s\n' %mean_matrix_pt2[i])
        text_file.write('Median:%s\n' %median_matrix_pt2[i])
        text_file.write('Standard Deviation:{0}\n' .format(std_matrix_pt2[i]))
        text_file.write('Variance:%s\n' %var_matrix_pt2[i])
        text_file.write('Quantiles[0.25, 0.50, 0.75]:%s\n' \
              %pd.Series(model_pos_pt2[i]).quantile\
              ([.25, .5, .75]).values)
        quant_matrix_pt2.append(pd.Series(model_pos_pt2[i]).quantile([.25, .5, .75]).values)
        text_file.write('Min:%s\n' %min(model_pos_pt2[i]))
        text_file.write('Max:%s\n\n' %max(model_pos_pt2[i]))        

###### Dataframe that contains statistical info of all ideal models

In [ ]:
d_pt1    = {'Position': range(1, positions+1), 'Mean': mean_matrix_pt1, 'Median': median_matrix_pt1, 'Std_Dev': std_matrix_pt1,\
     'Variance': var_matrix_pt1, 'Quantile': quant_matrix_pt1} #variable to pass data 
cols_pt1 = ['Position', 'Mean', 'Median',  'Std_Dev', 'Variance', 'Quantile'] #column names
try:
    df_ideal_pt1   = pd.DataFrame(data = d_pt1) 
except:
    pass
df_ideal_pt1   = df_ideal_pt1[cols_pt1]

d_pt2    = {'Position': range(1, positions+1), 'Mean': mean_matrix_pt2, 'Median': median_matrix_pt2, 'Std_Dev': std_matrix_pt2,\
     'Variance': var_matrix_pt2, 'Quantile': quant_matrix_pt2} #variable to pass data 
cols_pt2 = ['Position', 'Mean', 'Median',  'Std_Dev', 'Variance', 'Quantile'] #column names
try:
    df_ideal_pt2   = pd.DataFrame(data = d_pt2) 
except:
    pass
df_ideal_pt2   = df_ideal_pt2[cols_pt2]

###### Statistical values of all the ideal models in a textfile 

In [ ]:
with open("All_Model_Statistical_Output_pt1.txt", "w") as text_file:
    text_file.write('###########################\n')
    text_file.write('  STATISTICAL INFORMATION  \n')
    text_file.write('###########################\n')
    for i in range(0, positions):
        text_file.write('\nPosition %s\n'%(i+1))
        text_file.write('Obs       Mean        Median       Standard Deviation    Variance \
        Quantile[.25, .5, .75]\n')

        for j in range(0, observations):
            try:
                text_file.write('%s   %s  %s  %s    %s  %s\n'\
                                %((j+1), stats.mean(yout_full_pt1[i][j]), \
                                  stats.median(yout_full_pt1[i][j]), \
                                  stats.pstdev(yout_full_pt1[i][j]),\
                                  stats.variance(yout_full_pt1[i][j]),pd.Series(yout_full_pt1[i][j]).quantile([.25, .5, .75]).values))             
            except:
                continue
                
with open("All_Model_Statistical_Output_pt2.txt", "w") as text_file:
    text_file.write('###########################\n')
    text_file.write('  STATISTICAL INFORMATION  \n')
    text_file.write('###########################\n')
    for i in range(0, positions):
        text_file.write('\nPosition %s\n'%(i+1))
        text_file.write('Obs       Mean        Median       Standard Deviation    Variance \
        Quantile[.25, .5, .75]\n')

        for j in range(0, observations):
            try:
                text_file.write('%s   %s  %s  %s    %s  %s\n'\
                                %((j+1), stats.mean(yout_full_pt2[i][j]), \
                                  stats.median(yout_full_pt2[i][j]), \
                                  stats.pstdev(yout_full_pt2[i][j]),\
                                  stats.variance(yout_full_pt2[i][j]),pd.Series(yout_full_pt2[i][j]).quantile([.25, .5, .75]).values))             
            except:
                continue

###### Statistical values of all the model timeseries in a dataframe 

In [ ]:
pos_matrix_pt1 = []
obs_matrix_pt1 = []
mean_matrix_pt1 = []
median_matrix_pt1 = []
std_matrix_pt1 = []
var_matrix_pt1 = []
quant_matrix_pt1 = []

for i in range(0, positions):
    for j in range(0, observations):
        try:
            pos_matrix_pt1.append(i+1)
            obs_matrix_pt1.append(j+1)
            mean_matrix_pt1.append(stats.mean(yout_full_pt1[i][j]))
            median_matrix_pt1.append(stats.median(yout_full_pt1[i][j]))
            std_matrix_pt1.append(stats.pstdev(yout_full_pt1[i][j]))
            var_matrix_pt1.append(stats.variance(yout_full_pt1[i][j]))
            quant_matrix_pt1.append(pd.Series(yout_full_pt1[i][j]).quantile([.25, .5, .75]).values)
        except:
            del pos_matrix_pt1[-1]
            del obs_matrix_pt1[-1]
            continue

d_pt1    = {'Position': pos_matrix_pt1, 'Observation': obs_matrix_pt1, 'Mean': mean_matrix_pt1, 'Median': median_matrix_pt1, 'Std_Dev': std_matrix_pt1,\
     'Variance': var_matrix_pt1, 'Quantile': quant_matrix_pt1}
cols_pt1 = ['Position', 'Observation', 'Mean', 'Median',  'Std_Dev', 'Variance', 'Quantile']
df_all_pt1   = pd.DataFrame(data = d_pt1)
df_all_pt1   = df_all_pt1[cols_pt1]

pos_matrix_pt2 = []
obs_matrix_pt2 = []
mean_matrix_pt2 = []
median_matrix_pt2 = []
std_matrix_pt2 = []
var_matrix_pt2 = []
quant_matrix_pt2 = []

for i in range(0, positions):
    for j in range(0, observations):
        try:
            pos_matrix_pt2.append(i+1)
            obs_matrix_pt2.append(j+1)
            mean_matrix_pt2.append(stats.mean(yout_full_pt2[i][j]))
            median_matrix_pt2.append(stats.median(yout_full_pt2[i][j]))
            std_matrix_pt2.append(stats.pstdev(yout_full_pt2[i][j]))
            var_matrix_pt2.append(stats.variance(yout_full_pt2[i][j]))
            quant_matrix_pt2.append(pd.Series(yout_full_pt2[i][j]).quantile([.25, .5, .75]).values)
        except:
            del pos_matrix_pt2[-1]
            del obs_matrix_pt2[-1]
            continue

d_pt2    = {'Position': pos_matrix_pt2, 'Observation': obs_matrix_pt2, 'Mean': mean_matrix_pt2, 'Median': median_matrix_pt2, 'Std_Dev': std_matrix_pt2,\
     'Variance': var_matrix_pt2, 'Quantile': quant_matrix_pt2}
cols_pt2 = ['Position', 'Observation', 'Mean', 'Median',  'Std_Dev', 'Variance', 'Quantile']
df_all_pt2   = pd.DataFrame(data = d_pt2)
df_all_pt2   = df_all_pt2[cols_pt2]

###### State Space Parameters of all the ideal models in a textfile

In [ ]:
with open("State_Space_Parameters_pt1.txt", "w") as text_file:
    text_file.write('###########################\n')
    text_file.write('  STATE SPACE PARAMETERS   \n')
    text_file.write('###########################\n')
    for i in range(0, positions):
        text_file.write('\nPosition %s\n'%(i+1))
        text_file.write('%s'%system_matrix_pt1[i])
        text_file.write('\n')
        
with open("State_Space_Parameters_pt2.txt", "w") as text_file:
    text_file.write('###########################\n')
    text_file.write('  STATE SPACE PARAMETERS   \n')
    text_file.write('###########################\n')
    for i in range(0, positions):
        text_file.write('\nPosition %s\n'%(i+1))
        text_file.write('%s'%system_matrix_pt2[i])
        text_file.write('\n')

###### Model Validation

In [ ]:
try:
    os.makedirs('Model_Validation/')
except OSError, e:
    if e.errno != os.errno.EEXIST:
        raise   
    pass
for i in range(0, positions):
    fig = plt.figure(figsize = (5,4))
    fig.suptitle('Position %s'%(i + 1), fontsize = 20, fontweight = 'bold')
    plt.plot(model_time_pt2[i], model_output_pt2[i], '--r', label = 'ideal pt2 model')
    plt.plot(model_time_pt1[i], model_output_pt1[i], '--b', label = 'ideal pt1 model')
    plt.legend()
    for j in range(0, observations):
        try:
            
            plt.tight_layout(rect=[0, 0.03, 1, 0.95])
            plt.plot(df_ist_soll[i][j] * m_factor_array[j]) 
        except:
            pass
    plt.savefig('Model_Validation/Position %s model.png'%(i+1))
